<a href="https://colab.research.google.com/github/charmerDark/quantum_svm_vqc_comparison/blob/main/hons_vqc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit

     |████████████████████████████████| 6.0MB 13.3MB/s 
     |████████████████████████████████| 17.9MB 230kB/s 
     |████████████████████████████████| 204kB 44.3MB/s 
     |████████████████████████████████| 215kB 42.4MB/s 
     |████████████████████████████████| 2.1MB 42.4MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 1.0MB 36.4MB/s 
     |████████████████████████████████| 194kB 53.2MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 614kB 40.9MB/s 
     |████████████████████████████████| 3.2MB 37.9MB/s 
     |████████████████████████████████| 5.5MB 43.8MB/s 
  Created wheel for qiskit: filename=qiskit-0.25.0-cp37-none-any.whl size=3032 sha256=bd9ed1854661fc5bbedca5b27eaadb43274beef9a59100643f76bf8183563671
  Stored in directory: /root/.cache/pip/wheels/09/6e/f1/a7eaab6e3943d749f1b257f462f60fbc402b0c8e870b555169
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py

In [3]:
import numpy as np

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal, ZFeatureMap,EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report

import time

seed = 10599
aqua_globals.random_seed = seed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.aqua_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>


In [4]:
class circuit_result():
  '''
  class to store details of each circuit and later pickle it
  '''
  def details(self):
    print("Circuit type is: \t",self.circuit_type)
    print("feature_map used is :\t",self.feature_map)
    print("The paulis used were: \t",self.paulis)
    print("The entanglement layout is \t",self.entanglement)
    print("The repititions of feature map: \t",self.reps)
    print("running time is : \t",self.running_time)
    print("Accuracy report: \n",accuracy)
    print("Circuit Depth: \t", self.depth)
    print("Number of operations \t",self.count_ops)


  def __init__(self,circuit_type,feature_map,time,accuracy,depth,count_ops,paulis,reps,entanglement):
    self.circuit_type=circuit_type
    self.feature_map=feature_map
    self.running_time=time
    self.accuracy=accuracy
    self.depth=depth
    self.count_ops=count_ops
    self.paulis=paulis
    self.reps=reps
    self.entanglement=entanglement

In [5]:
training_size=120
test_size=30
class_labels = [r'A', r'B', r'C']
data, target = datasets.load_iris(return_X_y=True)
sample_train, sample_test, label_train, label_test =train_test_split(data, target, test_size=30, random_state=42)
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)
training_input = {key: (sample_train[label_train == k, :])[:training_size]
                      for k, key in enumerate(class_labels)}
test_input = {key: (sample_test[label_test == k, :])[:test_size]
                  for k, key in enumerate(class_labels)}

In [ ]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=80)
var_form = TwoLocal(4, ['ry','rz'], 'crz', reps=3,entanglement='circular')
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/optimizers/optimizer.py:50: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit.algorithms.optimizers', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/classifiers/vqc.py:98: DeprecationWarning: The package qiskit.aqua.algorithms.classifiers is deprecated. It was moved/refactored to qiskit_machine_learning.algorithms.classifiers (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-machine-learning')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/vq_algorithm.py:72: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refac

training took  543.2600898742676
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



In [ ]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=80)
var_form = EfficientSU2(4,reps=3)
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

training took  406.8485155105591
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.73      0.80        11
           2       0.73      0.89      0.80         9

    accuracy                           0.87        30
   macro avg       0.87      0.87      0.87        30
weighted avg       0.88      0.87      0.87        30



In [ ]:
TwoLocal(4, ['ry','rz'], 'crz', reps=3,entanglement='sca',insert_barriers=True).draw()

┌──────────┐┌──────────┐ ░ ┌──────────┐                         »
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├─░─┤ RZ(θ[8]) ├─────■───────────────────»
     ├──────────┤├──────────┤ ░ └────┬─────┘┌────┴─────┐             »
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├─░──────┼──────┤ RZ(θ[9]) ├──────■──────»
     ├──────────┤├──────────┤ ░      │      └──────────┘┌─────┴─────┐»
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├─░──────┼──────────────────┤ RZ(θ[10]) ├»
     ├──────────┤├──────────┤ ░      │                  └───────────┘»
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├─░──────■───────────────────────────────»
     └──────────┘└──────────┘ ░                                      »
«                   ░ ┌───────────┐┌───────────┐ ░                           »
«q_0: ──────────────░─┤ RY(θ[12]) ├┤ RZ(θ[16]) ├─░────────────────────■──────»
«                   ░ ├───────────┤├───────────┤ ░                    │      »
«q_1: ──────────────░─┤ RY(θ[13]) ├┤ RZ(θ[17]) ├─░────────────────────┼──────»
«                   ░ ├───────────┤├───────────┤ ░ ┌───────────┐      │      »
«q_2: ──────■───────░─┤ RY(θ[14]) ├┤ RZ(θ[18]) ├─░─┤ RZ(θ[20]) ├──────┼──────»
«     ┌─────┴─────┐ ░ ├───────────┤├───────────┤ ░ └─────┬─────┘┌─────┴─────┐»
«q_3: ┤ RZ(θ[11]) ├─░─┤ RY(θ[15]) ├┤ RZ(θ[19]) ├─░───────■──────┤ RZ(θ[21]) ├»
«     └───────────┘ ░ └───────────┘└───────────┘ ░              └───────────┘»
«     ┌───────────┐              ░ ┌───────────┐┌───────────┐ ░              »
«q_0: ┤ RZ(θ[22]) ├──────────────░─┤ RY(θ[24]) ├┤ RZ(θ[28]) ├─░──────────────»
«     └─────┬─────┘┌───────────┐ ░ ├───────────┤├───────────┤ ░              »
«q_1: ──────■──────┤ RZ(θ[23]) ├─░─┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─░───────■──────»
«                  └─────┬─────┘ ░ ├───────────┤├───────────┤ ░ ┌─────┴─────┐»
«q_2: ───────────────────■───────░─┤ RY(θ[26]) ├┤ RZ(θ[30]) ├─░─┤ RZ(θ[32]) ├»
«                                ░ ├───────────┤├───────────┤ ░ └───────────┘»
«q_3: ───────────────────────────░─┤ RY(θ[27]) ├┤ RZ(θ[31]) ├─░──────────────»
«                                ░ └───────────┘└───────────┘ ░              »
«                  ┌───────────┐              ░ ┌───────────┐┌───────────┐
«q_0: ─────────────┤ RZ(θ[34]) ├──────■───────░─┤ RY(θ[36]) ├┤ RZ(θ[40]) ├
«                  └─────┬─────┘┌─────┴─────┐ ░ ├───────────┤├───────────┤
«q_1: ───────────────────┼──────┤ RZ(θ[35]) ├─░─┤ RY(θ[37]) ├┤ RZ(θ[41]) ├
«                        │      └───────────┘ ░ ├───────────┤├───────────┤
«q_2: ──────■────────────┼────────────────────░─┤ RY(θ[38]) ├┤ RZ(θ[42]) ├
«     ┌─────┴─────┐      │                    ░ ├───────────┤├───────────┤
«q_3: ┤ RZ(θ[33]) ├──────■────────────────────░─┤ RY(θ[39]) ├┤ RZ(θ[43]) ├
«     └───────────┘                           ░ └───────────┘└───────────┘

In [ ]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(4,reps=3)
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

training took  509.36247539520264
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.82      0.90        11
           2       0.82      1.00      0.90         9

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30



In [ ]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=120)
var_form = TwoLocal(4, ['ry','rz'], 'crz', reps=3,entanglement='circular')
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

training took  815.8849918842316
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [8]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=120)
var_form = TwoLocal(4, ['rz'], 'crz', reps=3,entanglement='circular')
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/optimizers/optimizer.py:50: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit.algorithms.optimizers', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/classifiers/vqc.py:98: DeprecationWarning: The package qiskit.aqua.algorithms.classifiers is deprecated. It was moved/refactored to qiskit_machine_learning.algorithms.classifiers (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-machine-learning')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/vq_algorithm.py:72: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refac

training took  661.326092004776
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.30      0.46        30
           2       0.00      0.00      0.00         0

    accuracy                           0.30        30
   macro avg       0.33      0.10      0.15        30
weighted avg       1.00      0.30      0.46        30



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
feature_map = ZFeatureMap(feature_dimension=4, reps=1)
optimizer = SPSA(maxiter=120)
var_form = TwoLocal(4, ['rx', 'rz'], 'crz', reps=3,entanglement='circular')
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start=time.time()
result = vqc.run(quantum_instance)
end= time.time()
print("training took ",end-start)
params=vqc.optimal_params
y_pred=vqc.predict(sample_test,quantum_instance,params=params)
accuracy=classification_report(y_pred[1],label_test)
print(accuracy)

training took  807.7532076835632
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.69      0.82        13
           2       0.64      1.00      0.78         7

    accuracy                           0.87        30
   macro avg       0.88      0.90      0.87        30
weighted avg       0.92      0.87      0.87        30

